# Bibliotecas

In [24]:
import json
import os.path
import io
import os
from typing import List

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload

# 1. Open json file

In [2]:
with open("/home/lucas/Downloads/ffhq-dataset-v2.json","r") as infile:
    load = json.load(infile)
    infile.close()

In [7]:
load['51']['image']['file_url']

'https://drive.google.com/uc?id=1tmArkzG-jTBy53254w174kcsJTM9T7Fc'

In [32]:
a = load['51']['image']['file_url'].find('id=')
load['51']['image']['file_url'][28:].replace('id=','')

'1tmArkzG-jTBy53254w174kcsJTM9T7Fc'

# 2. Google API

In [18]:
scopes = ["https://www.googleapis.com/auth/drive"]
credentials_path = "../client_secret.json"
creds = Credentials.from_authorized_user_file("token.json", scopes)

In [19]:
service = build("drive", "v3", credentials=creds)

In [20]:
request = service.files().get_media(fileId='1tmArkzG-jTBy53254w174kcsJTM9T7Fc')

In [25]:
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

In [27]:
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [28]:
with open('teste.png', "wb") as f:
    fh.seek(0)
    f.write(fh.read())

In [14]:
flow = InstalledAppFlow.from_client_secrets_file(credentials_path, scopes)
creds = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=976236962688-rlku7o58bsq3pvq3mlnsvp4v33d4pah9.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A45293%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=FXzNy94sa8rVVTIgF4IQv8jN5FDh3V&access_type=offline


In [15]:
with open("token.json", "w") as token:
    token.write(creds.to_json())

In [9]:


import io
import os
from typing import List
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload


def run(file_id: str, save_path: str, credentials_path: str, scopes: List[str]):
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", scopes)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, scopes)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    service = build("drive", "v3", credentials=creds)

    # Different files require different download method, and I don't know how to distinguish them at the moment, so I can only try another one after an error
    # Export：Docs Editors files
    # Get：files with binary content
    try:
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))
    except Exception:
        request = service.files().export(
            fileId=file_id, mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))

    with open(save_path, "wb") as f:
        fh.seek(0)
        f.write(fh.read())


if __name__ == "__main__":
    file_id = "1JI17N-NFAIOxX_2Y88gmuKMlsuGhBPcB"
    save_path = "Sample Spreadsheet.xlsx"
    scopes = ["https://www.googleapis.com/auth/drive"]
    credentials_path = "credentials.json"
    run(file_id, save_path, credentials_path, scopes)




In [10]:
load['3242'].keys()

dict_keys(['category', 'metadata', 'image', 'thumbnail', 'in_the_wild'])

In [16]:
load['3242']

{'category': 'training',
 'metadata': {'photo_url': 'https://www.flickr.com/photos/7952984@N02/11026347713/',
  'photo_title': '',
  'author': 'suran2007',
  'country': '',
  'license': 'Attribution License',
  'license_url': 'https://creativecommons.org/licenses/by/2.0/',
  'date_uploaded': '2013-11-24',
  'date_crawled': '2018-10-10'},
 'image': {'file_url': 'https://drive.google.com/uc?id=16umbkObKDcEC9rWTExiUliBKkTF45SJy',
  'file_path': 'images1024x1024/03000/03242.png',
  'file_size': 1510761,
  'file_md5': '1c02a4b4c3aeb9df63783c0840889084',
  'pixel_size': [1024, 1024],
  'pixel_md5': 'a0dd45f1bb7f513dea41d50da42a6596',
  'face_landmarks': [[376.21, 497.23],
   [361.15, 552.88],
   [354.52, 610.86],
   [367.46, 668.29],
   [389.01, 729.67],
   [408.78, 804.34],
   [421.9, 878.1],
   [440.58, 943.08],
   [498.92, 967.65],
   [589.74, 960.81],
   [690.79, 928.29],
   [792.19, 884.29],
   [885.51, 826.47],
   [949.03, 750.71],
   [965.88, 652.35],
   [960.29, 558.12],
   [951.29, 

In [24]:
data

<Response [200]>

In [23]:
import requests

data = requests.get(load['3242']['image']['file_url'])
with open("tmp.png","wb") as outfile:
    #for content in data.content:
    outfile.write(data.content)
    outfile.close()

In [21]:
with open("tmp","wb") as outfile:
    #for content in data.content:
    outfile.write(data.content)
    outfile.close()